In [ ]:
#!pip install --upgrade git+https://github.com/pykeen/pykeen.git@master

In [ ]:
!pip install pykeen #pandas numpy scikit-learn matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import torch
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.evaluation import RankBasedEvaluator
from pykeen import predict
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
############################################
# 1. Определение устройства: GPU или CPU
############################################
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Используем устройство:", device)

In [ ]:
############################################
# 2. Загрузка данных и создание TriplesFactory
############################################
df = pd.read_csv('drive/MyDrive/knowledge_graph/kge_triples_all.csv')
print("Количество троек в датасете:", df.shape[0])

# Преобразуем DataFrame в массив троек
triples = df[['subject', 'predicate', 'object']].to_numpy()

# Создаём TriplesFactory (формируются entity_to_id и relation_to_id)
tf = TriplesFactory.from_labeled_triples(triples)
print("Количество уникальных сущностей:", len(tf.entity_to_id))
print("Количество уникальных отношений:", len(tf.relation_to_id))

In [ ]:
############################################
# 3. 80% train, 10% validation, 10% test
############################################
train_tf, valid_tf, test_tf = tf.split([0.8, 0.1, 0.1], random_state=42)
print("Тройки в обучающем наборе:", train_tf.num_triples)
print("Тройки в валидационном наборе:", valid_tf.num_triples)
print("Тройки в тестовом наборе:", test_tf.num_triples)

In [ ]:
############################################
# 4. Обучение модели TransE/ComplEx/RotatE
############################################
result = pipeline(
    training=train_tf,
    validation=valid_tf,
    testing=test_tf,
    model="RotatE",  # Выбираем модель TransE/ComplEx/RotatE
    model_kwargs=dict(embedding_dim=400),
    training_kwargs=dict(num_epochs=100, batch_size=128),
    optimizer="adam",
    optimizer_kwargs=dict(lr=0.001),
    random_seed=42,
    device=device,
)

In [ ]:
# Выводим агрегированные метрики из pipeline
print("Результаты pipeline (метрики):")
result.metric_results.to_df()

In [ ]:
############################################
# 5. Оценка модели с использованием RankBasedEvaluator для модели RotatE
############################################
evaluator = RankBasedEvaluator(filtered=True)
metrics_result = evaluator.evaluate(
    model=result.model,
    mapped_triples=test_tf.mapped_triples,
    additional_filter_triples=[train_tf.mapped_triples, valid_tf.mapped_triples],
)

print("Оценка модели RankBasedEvaluator:")
print("Mean Rank:", metrics_result.get_metric("mean_rank"))
print("Mean Reciprocal Rank (MRR):", metrics_result.get_metric("mean_reciprocal_rank"))
print("Hits@1:", metrics_result.get_metric("hits_at_1"))
print("Hits@10:", metrics_result.get_metric("hits_at_10"))

In [ ]:
#################################################
# 9. Сохранение обученной модели и ее мета-данных
#################################################

result.save_to_directory("drive/MyDrive/knowledge_graph/RotatE_model")
print("Модель и метаданные сохранены")

In [ ]:
############################################
# 7. Кластеризация
############################################

# Получаем эмбеддинги сущностей
entity_embeddings_tensor = result.model.entity_representations[0]()
entity_embeddings_tensor = entity_embeddings_tensor.detach().cpu()

print("Тип данных эмбеддингов:", entity_embeddings_tensor.dtype)
#print("Пример эмбеддинга:", entity_embeddings_tensor[0])

entity_embeddings_real = entity_embeddings_tensor.real.numpy()
entity_embeddings_imag = entity_embeddings_tensor.imag.numpy()

entity_embeddings_combined = np.concatenate([entity_embeddings_real, entity_embeddings_imag], axis=1)
print("Размерность после объединения:", entity_embeddings_combined.shape)

In [ ]:
pca = PCA(n_components=0.95)
embeddings_pca = pca.fit_transform(entity_embeddings_combined)

print("Размерность после PCA:", embeddings_pca.shape)

In [ ]:
# Получаем отображение ID в Название сущности
id_to_entity = {v: k for k, v in train_tf.entity_to_id.items()}
entities = [id_to_entity[i] for i in range(len(id_to_entity))] # список ID сущностей

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=42)  # эти кластеры на картинке ничего не означают, их выделил к-средних, надо красить по типам (белок, пептид, днк, рнк и тд)
clusters = kmeans.fit_predict(embeddings_pca)

df_plot = pd.DataFrame({
    "entity": entities,
    "PC1": embeddings_pca[:, 0],
    "PC2": embeddings_pca[:, 1],
    "cluster": clusters
})


plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_plot, x="PC1", y="PC2", hue="cluster", palette="deep")
plt.title("Кластеризация эмбеддингов сущностей (PCA)")
plt.xlabel("Компонента 1")
plt.ylabel("Компонента 2")
plt.legend(title="Кластер")
plt.show()

In [ ]:
############################################
# 8. Информация о результатах кластеризации
############################################
df_clusters = pd.DataFrame({
    "entity": entities,
    "cluster": clusters
})

# сколько объектов в каждом кластере
cluster_counts = df_clusters['cluster'].value_counts().sort_index()
print(f"Количество объектов в каждом кластере: {cluster_counts}")
print('========================')

# примеры из каждого кластера
print(" === Примеры сущностей в кластерах: === ")
for cluster_id in sorted(df_clusters['cluster'].unique()):
    cluster_entities = df_clusters[df_clusters['cluster'] == cluster_id]['entity']
    print(f"Кластер {cluster_id} (всего {len(cluster_entities)} сущностей):")
    print(cluster_entities.sample(min(10, len(cluster_entities))).tolist())  # случайные 10
    print('================================================================')